In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler
from math import ceil
from itertools import cycle
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from sklearn.model_selection import train_test_split
# used to provide a value to data that can be then used in equations, eg. Friday = 1 etc.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import style
style.use('ggplot')
from googletrans import Translator

from xgboost import XGBRegressor
from xgboost import plot_importance

import time
import sys
import gc
import pickle

In [2]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [77]:
#Read in .csv files into pandas dataframes
train       = pd.read_csv('sales_train.csv')
test        = pd.read_csv('test.csv').set_index('ID')
submission  = pd.read_csv('sample_submission.csv')
items       = pd.read_csv('items.csv')
item_cats   = pd.read_csv('item_categories.csv')
shops       = pd.read_csv('shops.csv')
shops_t     = pd.read_csv('shops_translated.csv')
items_t     = pd.read_csv('items_translated_text.csv')

In [78]:
#Down casts the data entries from int64 to int32 and float64 to float32
#This reduces the size of the records by almost half. (From 134mb to 61mb)
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [79]:
#Calls the downcasting function
train      = downcast_dtypes(train)
test       = downcast_dtypes(test)
submission = downcast_dtypes(submission)
items      = downcast_dtypes(items)
items_t    = downcast_dtypes(items_t)
item_cats  = downcast_dtypes(item_cats)
shops      = downcast_dtypes(shops)
shops_t    = downcast_dtypes(shops_t)

In [80]:
#Adds item_category_id to train dataframe and drops the item_name column
#Formats the date from string to datetime object
train = train.merge(items, on='item_id')
train = train.drop(columns = 'item_name')
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')

In [81]:
#Removes outlier price items from train dataframe
train = train[train.item_price < 90000]
train = train[train.item_cnt_day < 999]

#replaces the one instance of a negative price item 
#with the median item_price of the item with the id of 2973 and in shop id 32
median = train[(train.shop_id==32)
               &(train.item_id==2973)
               &(train.date_block_num==4)
               &(train.item_price>0)].item_price.median()

In [82]:
#Moves item_cnt_day column to the last column
train_cnt = train['item_cnt_day']
train.drop(labels=['item_cnt_day'], axis=1, inplace = True)
train.insert(6, 'item_cnt_day', train_cnt)

In [83]:
#Sums the item_cnt_day by date_block_month
#Drops the date column
#resets the index to start at 0 again
train_grouped_month = pd.DataFrame(train.groupby(['date_block_num', 'shop_id', 
                                                  'item_category_id', 'item_id',
                                                  'item_price'])['item_cnt_day'].sum().reset_index())

In [84]:
#renames item_cnt_day to item_cnt_month
train_grouped_month.rename(columns = {'item_cnt_day':'item_cnt_month'}, inplace = True)

In [85]:
train_grouped_month

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_month
0,0,0,2,5572,1322.0,10.0
1,0,0,2,5573,560.0,1.0
2,0,0,2,5575,806.0,4.0
3,0,0,2,5576,2231.0,5.0
4,0,0,2,5609,2381.0,1.0
...,...,...,...,...,...,...
1739014,33,59,79,17717,1250.0,4.0
1739015,33,59,79,17717,1999.0,1.0
1739016,33,59,83,22087,119.0,6.0
1739017,33,59,83,22088,119.0,2.0


In [70]:
#Converts all categorical features into strings because
#nominal intergers can not be converted to binary encoding
train_grouped_month_encoded = train_grouped_month.copy(deep=True)
train_grouped_month_encoded['date_block_num']   = [('month ' + str(i)) for i in train_grouped_month_encoded['date_block_num']]
train_grouped_month_encoded['shop_id']          = [('shop ' + str(i)) for i in train_grouped_month_encoded['shop_id']]
train_grouped_month_encoded['item_category_id'] = [('item_category ' + str(i)) for i in train_grouped_month_encoded['item_category_id']]
train_grouped_month_encoded['item_id']          = [('item ' + str(i)) for i in train_grouped_month_encoded['item_id']]

In [71]:
train_grouped_month_encoded

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_month
0,month 0,shop 0,item_category 2,item 5572,1322.0,10.0
1,month 0,shop 0,item_category 2,item 5573,560.0,1.0
2,month 0,shop 0,item_category 2,item 5575,806.0,4.0
3,month 0,shop 0,item_category 2,item 5576,2231.0,5.0
4,month 0,shop 0,item_category 2,item 5609,2381.0,1.0
...,...,...,...,...,...,...
1739014,month 33,shop 59,item_category 79,item 17717,1250.0,4.0
1739015,month 33,shop 59,item_category 79,item 17717,1999.0,1.0
1739016,month 33,shop 59,item_category 83,item 22087,119.0,6.0
1739017,month 33,shop 59,item_category 83,item 22088,119.0,2.0


In [72]:
train_grouped_month

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_month
0,0,0,2,5572,1322.0,10.0
1,0,0,2,5573,560.0,1.0
2,0,0,2,5575,806.0,4.0
3,0,0,2,5576,2231.0,5.0
4,0,0,2,5609,2381.0,1.0
...,...,...,...,...,...,...
1739014,33,59,79,17717,1250.0,4.0
1739015,33,59,79,17717,1999.0,1.0
1739016,33,59,83,22087,119.0,6.0
1739017,33,59,83,22088,119.0,2.0


In [73]:
train_grouped_month.to_csv('grouped_train_not_encoded.csv')

In [91]:
one_shop = train_grouped_month[train_grouped_month['shop_id'] == 0]

In [94]:
one_shop

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_month
0,0,0,2,5572,1322.0,10.0
1,0,0,2,5573,560.0,1.0
2,0,0,2,5575,806.0,4.0
3,0,0,2,5576,2231.0,5.0
4,0,0,2,5609,2381.0,1.0
...,...,...,...,...,...,...
69567,1,0,82,13357,21.0,1.0
69568,1,0,83,22087,44.0,19.0
69569,1,0,83,22088,44.0,18.0
69570,1,0,83,22091,63.0,1.0


In [111]:
valid_items = one_shop.item_id.unique()

In [112]:
valid_shops = train_grouped_month.shop_id.unique()

In [99]:
valid_items = list(valid_items)

In [100]:
valid_items

[5572,
 5573,
 5575,
 5576,
 5609,
 5612,
 5623,
 5627,
 5629,
 5630,
 5632,
 5635,
 5641,
 5643,
 13510,
 13517,
 13071,
 1956,
 1958,
 5680,
 13101,
 13102,
 5578,
 5580,
 5581,
 5583,
 5587,
 5588,
 5591,
 5592,
 5594,
 5601,
 5605,
 7882,
 7890,
 7892,
 7893,
 7894,
 7895,
 7907,
 7954,
 7956,
 11305,
 6672,
 13435,
 13478,
 13480,
 13482,
 13484,
 13487,
 13490,
 6668,
 6665,
 13425,
 13430,
 13431,
 4861,
 7905,
 13393,
 1409,
 1471,
 1595,
 1790,
 1836,
 1870,
 1985,
 2253,
 2281,
 2303,
 2307,
 2437,
 2649,
 2680,
 2682,
 2704,
 2715,
 2748,
 2760,
 2835,
 2846,
 2973,
 3045,
 3056,
 3076,
 3187,
 3253,
 3316,
 3320,
 3321,
 3366,
 3428,
 3433,
 3541,
 3686,
 3705,
 3850,
 3851,
 3852,
 3869,
 3877,
 4227,
 4229,
 4266,
 4281,
 4372,
 4464,
 4474,
 4476,
 4482,
 4486,
 4532,
 4792,
 4807,
 4810,
 4814,
 4815,
 4886,
 4887,
 4906,
 4960,
 5033,
 5256,
 5261,
 5273,
 5283,
 5918,
 5992,
 6072,
 6077,
 6122,
 6143,
 6154,
 6326,
 6402,
 6452,
 6612,
 6659,
 6681,
 6700,
 6725,
 67

In [108]:
def convert_list_to_options_dict(valid_items):
    list_of_dicts =[]
    
    for item in valid_items:
        temp_dict = {'label': item, 'value': item}
        list_of_dicts.append(temp_dict)
    return list_of_dicts

In [ ]:
def get_valid_item_list(train_grouped_month, shopid):
    one_shop = train_grouped_month[train_grouped_month['shop_id'] == shopid]
    return one_shop.item_id.unique()

In [109]:
valid_dict_list = convert_list_to_options_dict(valid_items)

In [113]:
valid_dict_list_shops = convert_list_to_options_dict(valid_shops)

In [114]:
valid_dict_list_shops

[{'label': 0, 'value': 0},
 {'label': 1, 'value': 1},
 {'label': 2, 'value': 2},
 {'label': 3, 'value': 3},
 {'label': 4, 'value': 4},
 {'label': 6, 'value': 6},
 {'label': 7, 'value': 7},
 {'label': 8, 'value': 8},
 {'label': 10, 'value': 10},
 {'label': 12, 'value': 12},
 {'label': 13, 'value': 13},
 {'label': 14, 'value': 14},
 {'label': 15, 'value': 15},
 {'label': 16, 'value': 16},
 {'label': 18, 'value': 18},
 {'label': 19, 'value': 19},
 {'label': 21, 'value': 21},
 {'label': 22, 'value': 22},
 {'label': 23, 'value': 23},
 {'label': 24, 'value': 24},
 {'label': 25, 'value': 25},
 {'label': 26, 'value': 26},
 {'label': 27, 'value': 27},
 {'label': 28, 'value': 28},
 {'label': 29, 'value': 29},
 {'label': 30, 'value': 30},
 {'label': 31, 'value': 31},
 {'label': 32, 'value': 32},
 {'label': 35, 'value': 35},
 {'label': 37, 'value': 37},
 {'label': 38, 'value': 38},
 {'label': 41, 'value': 41},
 {'label': 42, 'value': 42},
 {'label': 43, 'value': 43},
 {'label': 44, 'value': 44},
 

In [110]:
valid_dict_list

[{'label': 5572, 'value': 5572},
 {'label': 5573, 'value': 5573},
 {'label': 5575, 'value': 5575},
 {'label': 5576, 'value': 5576},
 {'label': 5609, 'value': 5609},
 {'label': 5612, 'value': 5612},
 {'label': 5623, 'value': 5623},
 {'label': 5627, 'value': 5627},
 {'label': 5629, 'value': 5629},
 {'label': 5630, 'value': 5630},
 {'label': 5632, 'value': 5632},
 {'label': 5635, 'value': 5635},
 {'label': 5641, 'value': 5641},
 {'label': 5643, 'value': 5643},
 {'label': 13510, 'value': 13510},
 {'label': 13517, 'value': 13517},
 {'label': 13071, 'value': 13071},
 {'label': 1956, 'value': 1956},
 {'label': 1958, 'value': 1958},
 {'label': 5680, 'value': 5680},
 {'label': 13101, 'value': 13101},
 {'label': 13102, 'value': 13102},
 {'label': 5578, 'value': 5578},
 {'label': 5580, 'value': 5580},
 {'label': 5581, 'value': 5581},
 {'label': 5583, 'value': 5583},
 {'label': 5587, 'value': 5587},
 {'label': 5588, 'value': 5588},
 {'label': 5591, 'value': 5591},
 {'label': 5592, 'value': 5592},
